# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install -i https://test.pypi.org/simple/ qurry
```

## 1.1 Setup Environment and Creat experiment executor

In [1]:
from qurry import EntropyMeasure, BackendWrapper
from qiskit import QuantumCircuit

- Before we start, we need to import the package of Qurry and other packages we need. 

- In Qurry, we provider a simple way to import backend called `BackendWrapper`, it will help us to import the backend we need, if your environment includes the GPU acceleration, `qiskit-aer-gpu`, it will be check the availability to access, if does, the GPU backend will also be available.

- The following chapter, we will told you how to use `BackendWrapper` to import the real backend from your IBM account by `IBMProvider` or `AccountProvider`, and the more powerful class `backendManager` to import the real backend more easily.

- In this example, we will use normal `aer_simulator` as our backend


In [2]:
backend = BackendWrapper()
backend('aer')

AerSimulator('aer_simulator_gpu')

In [3]:
print("| Does we have GPU backend to access:", backend.is_aer_gpu)
try:
    # If you have GPU, you can use this backend.
    print(backend('aer_gpu'))
except:
    pass

| Does we have GPU backend to access: True
aer_simulator_gpu


- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [4]:
experiment_execution_01 = EntropyMeasure(method='hadamard')
print(f'| The first method is which is `Hadamard Test`".')
experiment_execution_02 = EntropyMeasure()
print(f'| The default method is which is `Randomized Measure` we will use.')

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuit

- We use `TrivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0,8,2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we can add them to our `Qurry` object.
The attribute `.waves` of `Qurry` object is a customized `dict`, we can check it by print it.

In [7]:
experiment_execution_02.add(
    wave=sample01,              # The circuit object
    key='TrivialParamagnet',    # The name of the circuit
    replace=False               # True for if you want to replace the circuit with the same name.
)                               # Default is False, so if you add the same name when False, 
                                # it will be added with serial number.
experiment_execution_02.add(sample02, 'GHZ')
experiment_execution_02.add(sample03)
print(experiment_execution_02.waves)

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

- We can use `.measure()` to excute the specific circuit in `EntropyMeasure` object, and it will return hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- For `EntropyMeasure`, it will require the following parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurement
  - shots: the number of shots for each measurement. Default is 1024

In [8]:
exp1 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
)
exp1 

'16c7454d-3db3-413c-93c1-2e0a243750d2'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`

In [9]:
exp2 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
    backend=backend('aer')
)
exp2

'7e95c263-8185-43b3-aad5-718f5ba1cf40'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.

In [10]:
print(experiment_execution_02.exps)

<ExperimentContainer={
    16c7454d-3db3-413c-93c1-2e0a243750d2: ...
    7e95c263-8185-43b3-aad5-718f5ba1cf40: ...
} with 2 experiments load, a customized dictionary>


- The following is the other informatiom you can access.

In [11]:
experiment_execution_02.exps[exp1]

<qurrentRandomized.Experiment with exp_id=16c7454d-3db3-413c-93c1-2e0a243750d2, EntropyRandomizedArguments(exp_name='w=TrivialParamagnet.with100random.qurrent_haar', times=100, measure=(0, 8), unitary_loc=(0, 8), workers_num=16), Commonparams(exp_id='16c7454d-3db3-413c-93c1-2e0a243750d2', wave_key='TrivialParamagnet', shots=1024, backend=<AerSimulator('aer_simulator')>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes={'build': '2024-03-19 19:05:45', 'run': '2024-03-19 19:05:45'}), 0 unused arguments, 6 preparing dates, 2 experiment result datasets, and 0 analysis>

In [12]:
print(experiment_execution_02.exps[exp1].statesheet())

 # qurrentRandomized.Experiment with exp_id=16c7454d-3db3-413c-93c1-2e0a243750d2
 - arguments
   - exp_name ---------------- w=TrivialParamagnet.with100random.qurrent_haar
   - times ------------------- 100
   - measure ----------------- (0, 8)
   - unitary_loc ------------- (0, 8)
   - workers_num ------------- 16
 - commonparams
   - exp_id ------------------ 16c7454d-3db3-413c-93c1-2e0a243750d2   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - wave_key ---------------- TrivialParamagnet
   - shots ------------------- 1024
   - backend ----------------- aer_simulator
   - run_args ---------------- {}
   - transpile_args ---------- {}
   - tags -------------------- ()
   - default_analysis -------- []
   - save_location ----------- .
   - filename ---------------- 
   - files ------------------- {}
   - serial ------------------ None
   - summoner_id ------------- None
   - summoner_name ----------- None
   - datetimes --------------- {'build': '2

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

- We want to know the Renyi Entropy of half system. Then we can use `.analyze()` to calculate it and it will store a `Analysis` object in `Experiment`.
- The following is the parameter `analyze` of `EntropyMeasure` required.
  - degree: The size or range of subsystem, if it is a number, it will be the size of subsystem, if it is a tuple, it will be the range of subsystem.

In [13]:
exp1_report1 = experiment_execution_02.exps[exp1].analyze(
    degree=(0, 4)
)
exp1_report2 = experiment_execution_02.exps[exp1].analyze(
    degree=4
)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.

In [14]:
print(exp1_report1)
print(exp1_report1.header)

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.1818041610717773, entropy=-0.2409909836888523, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-03-19 19:05:47', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.

In [15]:
print(experiment_execution_02.exps[exp1].reports)
print(experiment_execution_02.exps[exp1].reports[0])
print(experiment_execution_02.exps[exp1].reports[0].header)

<AnalysisContainer={
    0: ...
    1: ...
} with 2 analysis load, a customized dictionary>
<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.1818041610717773, entropy=-0.2409909836888523, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-03-19 19:05:47', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.

In [16]:
main, side_prodct = experiment_execution_02.exps[exp1].reports[0].export()

- And there is it

In [17]:
main

{'purity': 1.1818041610717773,
 'entropy': -0.2409909836888523,
 'puritySD': 1.1137335938579271,
 'entropySD': 1.3595974575627563,
 'bitStringRange': [0, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.4394626808166504,
 'entropyAllSys': -0.5255303864014051,
 'puritySDAllSys': 1.7930460028623025,
 'entropySDAllSys': 1.7970723457364257,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.2004940915634181,
 'mitigatedPurity': 0.8391548602748656,
 'mitigatedEntropy': 0.2529910201765524,
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.000714156,
 'takingTimeAllSys': 0.003152348,
 'input': {'degree': [0, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-03-19 19:05:47',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [18]:
side_prodct

{'purityCells': {61: 1.3342533111572266,
  25: 4.869670867919922,
  29: 1.5328655242919922,
  65: 3.6087779998779297,
  70: 0.8376827239990234,
  35: 0.7242488861083984,
  72: 1.8999004364013672,
  16: 0.35748291015625,
  56: 0.1423511505126953,
  85: 2.0770034790039062,
  20: 0.9743423461914062,
  98: 0.5528850555419922,
  93: 3.852611541748047,
  50: 0.8923282623291016,
  45: 0.7859325408935547,
  28: 0.2659645080566406,
  62: 0.2904834747314453,
  74: 1.1436920166015625,
  81: 0.9935340881347656,
  87: 0.31452178955078125,
  94: 0.8372821807861328,
  38: 0.64312744140625,
  59: 1.2306499481201172,
  64: 0.19817543029785156,
  9: 0.1980152130126953,
  76: 0.5458011627197266,
  77: 4.975500106811523,
  97: 0.6272773742675781,
  4: 1.0198554992675781,
  54: 1.0745353698730469,
  84: 0.8149662017822266,
  11: 5.82179069519043,
  1: 0.5531768798828125,
  24: 2.1800804138183594,
  40: 2.3923168182373047,
  37: 0.2369251251220703,
  86: 0.4221477508544922,
  32: 1.0298175811767578,
  91: 0

In [19]:
experiment_execution_02.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-03-19 19:05:47
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (0, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 1.1818041610717773
   - entropy ---------------------- -0.2409909836888523
   - puritySD --------------------- 1.1137335938579271
   - entropySD -------------------- 1.3595974575627563
   - purityCells ------------------ {61: 1.3342533111572266, 25: 4.869670867919922, 29: 1.5328655242919922, 65: 3.6087779998779297, 70: 0.8376827239990234, 35: 0.7242488861083984, 72: 1.8999004364013672, 16: 0.35748291015625, 56: 0.1423511505126953, 85: 2.0770034790039062, 20: 0.9743423461914062, 98: 0.5528850555419922, 93: 3.

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.

In [20]:
from qurry.process import AVAIBILITY_STATESHEET
print(AVAIBILITY_STATESHEET)

 | Qurry version: 0.7.1.dev5
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability           Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   None  
   - echo_cell .................... True   True   None  
 - utils
   - randomized ................... True   True   True  
   - construct .................... True   None   True  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported yet.
--------------------------------------------------------

